#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #2
# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The $\lambda$ Return

## Description

Given an MDP and a particular time step $t$ of a task (continuing or
episodic), the $\lambda$-return, $G_t^\lambda$, $0\leq\lambda\leq 1$, is
a weighted combination of the $n$-step returns $G_{t:t+n}$, $n \geq 1$:

$${G_t^\lambda = \sum\limits_{n=1}^\infty(1-\lambda)\lambda^{n-1}G_{t:t+n}.}$$

While the $n$-step return $G_{t:t+n}$ can be viewed as the target of
an $n$-step TD update rule, the $\lambda$-return can be viewed as the
target of the update rule for the TD$(\lambda)$ prediction algorithm,
which you will become familiar with in project 1.

Consider the Markov reward process described by the following state
diagram and assume the agent is in state $0$ at time $t$ (also assume
the discount rate is $\gamma=1$). A Markov reward process can be thought of as
an MDP with only one action possible from each state (denoted as action $0$ in
the figure below).

![image](https://d1b10bmlvqabco.cloudfront.net/paste/jqmfo7d3watba/9e6fd83672f880704b8418728297fc077786c8907d87fec631601da9ff4c85ef/hw2.png)

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   You will be given `p`, the probability of transitioning from state
    $0$ to state $1$, `V`, the estimate of the value function at time
    $t$, represented as a vector
    $[V(0), V(1), V(2), V(3), V(4), V(5), V(6)]$, and `rewards`, a
    vector of the rewards $[r_0, r_1, r_2, r_3, r_4, r_5, r_6]$
    corresponding to the MDP.

-   Your return value should be a value of $\lambda$,
    strictly less than 1, such that the expected value of the
    $\lambda$-return equals the expected Monte-Carlo return at time $t$.

-   Your answer must be correct to $3$ decimal places, truncated (e.g.
    3.14159265 becomes 3.141).

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 3: TD and Friends

-   Chapter 7 (7.1 $n$-step TD Prediction) and Chapter 12 (12.1 The
    $\lambda$-return) of
    http://incompleteideas.net/book/the-book-2nd.html

-   'Learning to Predict by the Method of Temporal Differences', R.
    Sutton, 1988

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions -
    only the last score is kept.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [16]:
################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np
import math

class TDAgent(object):
    def __init__(self):
        pass

    def validate_inputs(self, p, V, rewards):
        are_inputs_valid = True
        if p < 0 or p > 1:
            are_inputs_valid = False
        if len(V) == 0:
            are_inputs_valid = False
        if len(rewards) == 0:
            are_inputs_valid = False
        return are_inputs_valid

    def fetch_rewards(self, rewards, sample_seq):
        if sample_seq == 1:
            episode = [rewards[0], rewards[2], rewards[4], rewards[5], rewards[6]]
        else:
            episode = [rewards[1], rewards[3], rewards[4], rewards[5], rewards[6]]
        return episode

    def fetch_value_funcs(self, V, sample_seq):
        if sample_seq == 1:
            episode = [V[0], V[1], V[3], V[4], V[5], V[6]]
        else:
            episode = [V[0], V[2], V[3], V[4], V[5], V[6]]
        return episode

    def calculate_expected_mc_return(self, p, rewards):
        return p * sum(self.fetch_rewards(rewards, 1)) + (1 - p) * sum(self.fetch_rewards(rewards, 2))

    def fetch_n_step_td_update(self, V, rewards, sample_seq, no_steps):
        state_value_funcs = self.fetch_value_funcs(V, sample_seq)
        episode_rewards = self.fetch_rewards(rewards, sample_seq)
        n_step_rewards = [episode_rewards[i] for i in range(no_steps)]
        value = sum(n_step_rewards) + state_value_funcs[no_steps]
        return value

    def calculate_n_step_td_return(self, p, V, rewards, no_steps):
        first_term = p * self.fetch_n_step_td_update(V, rewards, 1, no_steps)
        second_term = (1 - p) * self.fetch_n_step_td_update(V, rewards, 2, no_steps)
        return first_term + second_term

    def solve(self, p, V, rewards):
        if self.validate_inputs(p, V, rewards):
            n_step_returns = list()
            for i in range(1, 6):
                n_step_returns.append(self.calculate_n_step_td_return(p, V, rewards, i))
            expected_mc_return = self.calculate_expected_mc_return(p, rewards)
            n_step_returns.append(0)
            a = np.array(n_step_returns)
            n_step_returns1 = list()
            n_step_returns1.append(0)
            n_step_returns1.extend(n_step_returns[0:len(n_step_returns) - 1])
            b = np.array(n_step_returns1) * -1
            n_step_returns2 = list()
            n_step_returns2.extend([-expected_mc_return, 0, 0, 0, 0, n_step_returns[-2]])
            c = np.array(n_step_returns2)
            array_tuple = (a, b, c)
            arrays = np.vstack(array_tuple)
            coeff = arrays.sum(axis=0)
            roots = np.roots(coeff[::-1])
            real_roots = roots[np.isreal(roots)]
            real_parts = [np.real(rt) for rt in real_roots]
            real_positive_roots = [rt for rt in real_parts if rt > 0 and rt < 1]
            final_root_array = [rt1 for rt1 in real_positive_roots if abs(1 - rt1) > 1e-3]
            final_root = math.trunc(final_root_array[0] * 1000) * .001
            return final_root
        return 0.000

In [3]:
################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np
import math

class TDAgent(object):
    def __init__(self):
        pass

    def validate_inputs(self, p, V, rewards):
        are_inputs_valid = True
        if p < 0 or p > 1:
            are_inputs_valid = False
        if len(V) == 0:
            are_inputs_valid = False
        if len(rewards) == 0:
            are_inputs_valid = False
        return are_inputs_valid

    def fetch_rewards(self, rewards, sample_seq):
        if sample_seq == 1:
            episode = [rewards[0], rewards[2], rewards[4], rewards[5], rewards[6]]
        else:
            episode = [rewards[1], rewards[3], rewards[4], rewards[5], rewards[6]]
        return episode

    def fetch_value_funcs(self, V, sample_seq):
        if sample_seq == 1:
            episode = [V[0], V[1], V[3], V[4], V[5], V[6]]
        else:
            episode = [V[0], V[2], V[3], V[4], V[5], V[6]]
        return episode

    def calculate_expected_mc_return(self, p, rewards):
        return p * sum(self.fetch_rewards(rewards, 1)) + (1 - p) * sum(self.fetch_rewards(rewards, 2))

    def fetch_n_step_td_update(self, V, rewards, sample_seq, no_steps):
        state_value_funcs = self.fetch_value_funcs(V, sample_seq)
        episode_rewards = self.fetch_rewards(rewards, sample_seq)
        n_step_rewards = [episode_rewards[i] for i in range(no_steps)]
        value = sum(n_step_rewards) + state_value_funcs[no_steps]
        return value

    def calculate_n_step_td_return(self, p, V, rewards, no_steps):
        first_term = p * self.fetch_n_step_td_update(V, rewards, 1, no_steps)
        second_term = (1 - p) * self.fetch_n_step_td_update(V, rewards, 2, no_steps)
        return first_term + second_term

    def solve(self, p, V, rewards):
        if self.validate_inputs(p, V, rewards):
            n_step_returns = list()
            for i in range(1, 6):
                n_step_returns.append(self.calculate_n_step_td_return(p, V, rewards, i))
            expected_mc_return = self.calculate_expected_mc_return(p, rewards)
            n_step_returns.append(0)
            a = np.array(n_step_returns)
            n_step_returns1 = list()
            n_step_returns1.append(0)
            n_step_returns1.extend(n_step_returns[0:len(n_step_returns) - 1])
            b = np.array(n_step_returns1) * -1
            n_step_returns2 = list()
            n_step_returns2.extend([-expected_mc_return, 0, 0, 0, 0, n_step_returns[-2]])
            c = np.array(n_step_returns2)
            array_tuple = (a, b, c)
            arrays = np.vstack(array_tuple)
            coeff = arrays.sum(axis=0)
            roots = np.roots(coeff[::-1])
            real_roots = roots[np.isreal(roots)]
            real_parts = [np.real(rt) for rt in real_roots]
            real_positive_roots = [rt for rt in real_parts if rt > 0 and rt < 1]
            final_root_array = [rt1 for rt1 in real_positive_roots if abs(1 - rt1) > 1e-3]
            final_root = math.trunc(final_root_array[0] * 1000) * .001
            return final_root
        return 0.000

## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestTDNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.81,
                V=[0.0, 4.0, 25.7, 0.0, 20.1, 12.2, 0.0],
                rewards=[7.9, -5.1, 2.5, -7.2, 9.0, 0.0, 1.6]
            ),
            0.622,
            decimal=3
        )
        
    def test_case_2(self):
        agent = TDAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                p=0.22,
                V=[12.3, -5.2, 0.0, 25.4, 10.6, 9.2, 0.0],
                rewards=[-2.4, 0.8, 4.0, 2.5, 8.6, -6.4, 6.1]
            ),
            0.519,
            decimal=3
        )
        
    def test_case_3(self):
        agent = TDAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                p=0.64,
                V=[-6.5, 4.9, 7.8, -2.3, 25.5, -10.2, 0.0],
                rewards=[-2.4, 9.6, -7.8, 0.1, 3.4, -2.1, 7.9]
            ),
            0.207,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestTDNotebook) ... ok
test_case_2 (__main__.TestTDNotebook) ... ok
test_case_3 (__main__.TestTDNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK


## Test cases

We have provided some test cases for you to help verify your implementation.